201911181 이승민

## 임포트 및 세팅

In [1]:
import pyspark
import os
import json
import requests
import pandas as pd
import numpy as np
from pyspark.sql import Row
from pyspark.sql.functions import udf
from pyspark.sql import functions as F
from pyspark.sql.types import *

In [2]:
myConf = pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
.builder\
.master('local')\
.appName('myApp')\
.config(conf=myConf)\
.getOrCreate()

21/10/23 21:32:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
def getKey(keyPath):
    d = dict()
    f = open(keyPath, 'r')
    for line in f.readlines():
        row = line.split('=')
        row0 = row[0]
        d[row0] = row[1].strip()
    return d

# 문제 1-1의 답

In [12]:
base = 'http://openapi.seoul.go.kr:8088'
key = '65714f71456b726938364851437a44'
form = 'json'
service = 'InfoTrdarAptQq'
year = '2021'
start = '1'
end = '100'
url = '/'.join([base, key, form, service, start, end, year])
info = requests.get(url).json()
print(url)

http://openapi.seoul.go.kr:8088/65714f71456b726938364851437a44/json/InfoTrdarAptQq/1/100/2021


# 문제 1-2의 답

In [15]:
info['InfoTrdarAptQq']['row'][0]

{'STDR_YY_CD': '2021',
 'TRDAR_SE_CD': 'A',
 'TRDAR_SE_CD_NM': '골목상권',
 'APT_HSMP_CO': 5.0,
 'AE_66_SQMT_BELO_HSHLD_CO': 32.0,
 'AE_66_SQMT_HSHLD_CO': 0.0,
 'AE_99_SQMT_HSHLD_CO': 0.0,
 'AE_132_SQMT_HSHLD_CO': 0.0,
 'AE_165_SQMT_HSHLD_CO': 0.0,
 'PC_1_HDMIL_BELO_HSHLD_CO': 0.0,
 'PC_1_HDMIL_HSHLD_CO': 32.0,
 'PC_2_HDMIL_HSHLD_CO': 0.0,
 'PC_3_HDMIL_HSHLD_CO': 0.0,
 'PC_4_HDMIL_HSHLD_CO': 0.0,
 'PC_5_HDMIL_HSHLD_CO': 0.0,
 'PC_6_HDMIL_ABOVE_HSHLD_CO': 0.0,
 'AVRG_AE': 44.0,
 'AVRG_MKTC': 150247619.0,
 'STDR_QU_CD': '1',
 'TRDAR_CD': '1000975',
 'TRDAR_CD_NM': '동남로82길'}

# 문제 1-3의 답

In [27]:
rows = info['InfoTrdarAptQq']['row']
for (i, data) in enumerate(rows):
    print('{}\t{}\t{}\t{}\t{}'.format(data['TRDAR_SE_CD_NM'], data['APT_HSMP_CO'], data['PC_6_HDMIL_ABOVE_HSHLD_CO'], data['AVRG_MKTC'], data['TRDAR_CD_NM']))

골목상권	5.0	0.0	150247619.0	동남로82길
발달상권	3.0	0.0	161157476.0	당산역_2
발달상권	1.0	0.0	81750000.0	서울 성북구 안암역
전통시장	37.0	1.0	183067389.0	길동복조리시장
관광특구	71.0	87.0	409473123.0	이태원 관광특구
관광특구	8.0	271.0	237967427.0	동대문패션타운 관광특구
관광특구	5.0	0.0	112097903.0	종로·청계 관광특구
관광특구	22.0	1156.0	527347107.0	잠실 관광특구
골목상권	26.0	22.0	249322039.0	계동길
골목상권	5.0	0.0	87700490.0	난계로27길
골목상권	5.0	0.0	161118305.0	돈화문로11가길
골목상권	41.0	0.0	105457249.0	명륜길
골목상권	41.0	0.0	214023214.0	백석동길
골목상권	3.0	0.0	213246032.0	북촌로11길
골목상권	2.0	2.0	443000000.0	북촌로5길
골목상권	5.0	6.0	430007143.0	북촌로5나길
골목상권	1.0	2.0	572000000.0	삼청로5길
골목상권	12.0	0.0	152368531.0	성균관로5길
골목상권	19.0	0.0	141952840.0	숭인동길
골목상권	100.0	0.0	177741821.0	옥인길
골목상권	1.0	0.0	112279412.0	율곡로16길
골목상권	20.0	0.0	153940955.0	이화장길
골목상권	9.0	0.0	256264683.0	자하문로7길
골목상권	3.0	0.0	81284178.0	종로58가길
골목상권	87.0	0.0	124591950.0	종로65길
골목상권	15.0	0.0	185365873.0	지봉로8길
골목상권	8.0	0.0	142664918.0	창경궁로35길
골목상권	17.0	0.0	107096078.0	창신2길
골목상권	8.0	0.0	95141667.0	창신5길
골목상권	86.0	0.0	116533576.0	창신길
골목상권	50.0	0.0	99746000.0	통일로

# 문제 2

In [29]:
yesterday=[
    ["Yesterday 어제는"],
    ["All my troubles seemed to far away 내 모든 문제들이 참 멀리 있는 것 같았는데"],
    ["Now it looks as though they're here to stay 이제는 그 모든 문제들이 여기에 있는 것처럼 보여"],
    ["Oh, I believe in yesterday 오, 어제가 좋았는데"],
    ["Suddenly 갑자기 그-림-자-가 2021 그-림-자-가 2021"],
    ["I'm not half the man I used to be 난 예전 내 모습의 반도 못한 사람이 되어버렸어"],
    ["There's a shadow hanging over me 내 위에 그림자가 드리워져 있네"],
    ["Oh!, yesterday came suddenly 오!, 이제는 너무 갑자기 와버렸어.."],
]
# UPPER, 불용어: !, -, 2021
# 전부 계산한 뒤 빈도 1 초과의 한글자 키는 제거

# 문제 2-1의 답

In [48]:
rdd = spark.sparkContext.parallelize(yesterday)
corpus = rdd.flatMap(lambda x: x).flatMap(lambda line: line.split())

ans = corpus.map(lambda word: (word, 1))\
.groupByKey()\
.mapValues(sum)\
.sortByKey(True)\
.collect()
ans

[('2021', 2),
 ('All', 1),
 ('I', 2),
 ("I'm", 1),
 ('Now', 1),
 ('Oh!,', 1),
 ('Oh,', 1),
 ('Suddenly', 1),
 ("There's", 1),
 ('Yesterday', 1),
 ('a', 1),
 ('as', 1),
 ('away', 1),
 ('be', 1),
 ('believe', 1),
 ('came', 1),
 ('far', 1),
 ('half', 1),
 ('hanging', 1),
 ('here', 1),
 ('in', 1),
 ('it', 1),
 ('looks', 1),
 ('man', 1),
 ('me', 1),
 ('my', 1),
 ('not', 1),
 ('over', 1),
 ('seemed', 1),
 ('shadow', 1),
 ('stay', 1),
 ('suddenly', 1),
 ('the', 1),
 ("they're", 1),
 ('though', 1),
 ('to', 3),
 ('troubles', 1),
 ('used', 1),
 ('yesterday', 2),
 ('갑자기', 2),
 ('같았는데', 1),
 ('것', 1),
 ('것처럼', 1),
 ('그', 1),
 ('그-림-자-가', 2),
 ('그림자가', 1),
 ('난', 1),
 ('내', 3),
 ('너무', 1),
 ('되어버렸어', 1),
 ('드리워져', 1),
 ('멀리', 1),
 ('모든', 2),
 ('모습의', 1),
 ('못한', 1),
 ('문제들이', 2),
 ('반도', 1),
 ('보여', 1),
 ('사람이', 1),
 ('어제가', 1),
 ('어제는', 1),
 ('여기에', 1),
 ('예전', 1),
 ('오!,', 1),
 ('오,', 1),
 ('와버렸어..', 1),
 ('위에', 1),
 ('이제는', 2),
 ('있네', 1),
 ('있는', 2),
 ('좋았는데', 1),
 ('참', 1)]

# 문제 2-2의 답

In [55]:
ans_sorted = sorted(ans, key = lambda element: -element[1])
ans_second_final = list(filter(lambda x: x[1] > 1, ans_sorted))
ans_second_final

[('to', 3),
 ('내', 3),
 ('2021', 2),
 ('I', 2),
 ('yesterday', 2),
 ('갑자기', 2),
 ('그-림-자-가', 2),
 ('모든', 2),
 ('문제들이', 2),
 ('이제는', 2),
 ('있는', 2)]

# 문제 2-3의 답

In [88]:
first = corpus.map(lambda x: x.upper())
second = first.filter(lambda x: not x.isnumeric())
third = second.map(lambda x: x.translate(str.maketrans('!-', '  ')).replace(' ', ''))
ans = third.map(lambda word: (word, 1))\
.groupByKey()\
.mapValues(sum)\
.sortByKey(True)\
.collect()
ans_sorted = sorted(ans, key = lambda element: -element[1])
ans_second_final = list(filter(lambda x: x[1] > 1, ans_sorted))
ans_second_final

[('TO', 3),
 ('YESTERDAY', 3),
 ('그림자가', 3),
 ('내', 3),
 ('I', 2),
 ('OH,', 2),
 ('SUDDENLY', 2),
 ('갑자기', 2),
 ('모든', 2),
 ('문제들이', 2),
 ('오,', 2),
 ('이제는', 2),
 ('있는', 2)]

# 문제 2-4의 답

In [90]:
ans_third = list(filter(lambda x: len(x[0]) > 1, ans_second_final))
ans_third

[('TO', 3),
 ('YESTERDAY', 3),
 ('그림자가', 3),
 ('OH,', 2),
 ('SUDDENLY', 2),
 ('갑자기', 2),
 ('모든', 2),
 ('문제들이', 2),
 ('오,', 2),
 ('이제는', 2),
 ('있는', 2)]

# 문제 3

In [29]:
filename = '서울특별시 코로나19 자치구별 확진자 발생동향.json'
df= spark.read.json(filename)

# 문제 3-1의 답

In [30]:
print(df.count())
df = df.drop('_corrupt_record')
pdf = df.toPandas()
pdf = pdf.dropna()
df = spark.createDataFrame(pdf)

633


In [31]:
df.count()

172

# 문제 3-2의 답

In [47]:
newDf = df.withColumn('Date', df['jcg_dt'].cast(DateType()))
newDf.select(['jongno', 'mapo', 'yongsan', 'tot']).show()

+------+----+-------+------+
|jongno|mapo|yongsan|   tot|
+------+----+-------+------+
|  1968|4127|   3031|9126.0|


# 문제 3-3의 답

In [50]:
from pyspark.sql.types import *
import pyspark.sql.functions as F
perfect_df.filter(F.month(perfect_df['Date']) == '03').select[per'jongno']

TypeError: 'method' object is not subscriptable

# 문제 3-4의 답

In [49]:
perfect_df = newDf.withColumn('year', F.year('Date')).withColumn('month', F.month('Date'))
perfect_df.groupby('year').pivot('month').show()

+-----+-------+-------+
|month|   2020|   2021|
+-----+-------+-------+
|   07| 1273.0|62849.0|
+-----+-------+-------+
